In [3]:
%pwd

'c:\\Users\\Harsha Sai\\Downloads\\kidney-disease-classification-cnn\\research'

In [4]:
import os
os.chdir("../")
%pwd

'c:\\Users\\Harsha Sai\\Downloads\\kidney-disease-classification-cnn'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from kidney_classifier.constants import *
from ensure import ensure_annotations
from kidney_classifier.utils.common import create_directories, read_yaml
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    @ensure_annotations
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown
from kidney_classifier import logger
from kidney_classifier.utils.common import get_size

In [8]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> str:

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into {zip_download_dir}")
            
            file_id = dataset_url.split("/")[:-2]
            prefix = "https://drive.google.com/uc?export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Downloading data from {dataset_url} into {zip_download_dir}")
        
        except Exception as e:
            raise e
        
    def unzip_data(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as f:
            f.extractall(unzip_path)
            

            


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()

except Exception as e:
    raise e